In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-file/tested.csv


In [3]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer

In [4]:
df = pd.read_csv('/kaggle/input/test-file/tested.csv')[['Age', 'Fare','SibSp','Parch','Survived']]

In [5]:
df.dropna(inplace=True)

In [7]:
df.head()

,Age,Fare,SibSp,Parch,Survived
0,34.5,7.8292,0,0,0
1,47.0,7.0000,1,0,1
2,62.0,9.6875,0,0,0
3,27.0,8.6625,0,0,0
4,22.0,12.2875,1,1,1


In [8]:
df['Family'] = df['SibSp'] + df['Parch']

In [9]:
df.head()

,Age,Fare,SibSp,Parch,Survived,Family
0,34.5,7.8292,0,0,0,0
1,47.0,7.0000,1,0,1,1
2,62.0,9.6875,0,0,0,0
3,27.0,8.6625,0,0,0,0
4,22.0,12.2875,1,1,1,2


In [10]:
df.drop(columns=['SibSp' , 'Parch'], inplace = True)

In [11]:
df.head()

,Age,Fare,Survived,Family
0,34.5,7.8292,0,0
1,47.0,7.0000,1,1
2,62.0,9.6875,0,0
3,27.0,8.6625,0,0
4,22.0,12.2875,1,2


In [12]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

In [14]:
X_train.head()

,Age,Fare,Family
281,0.75,13.7750,2
96,76.00,78.8500,1
341,32.00,7.5792,0
18,27.00,7.9250,1
26,22.00,61.9792,1


In [15]:
# without binarization

clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.6268656716417911

In [16]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring = 'accuracy'))

0.595187165775401

In [17]:
# applying binarization

from sklearn.preprocessing import Binarizer

In [20]:
# yahan binarizer family col pr lagayenge. aur laga kr ye pta krne ki koshish krenge ki koi insaan akela travel kr rha hai ki nhi

trf = ColumnTransformer([
    ('bin', Binarizer(copy = False), ['Family'])
], remainder ='passthrough')

In [21]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [22]:
pd.DataFrame(X_train_trf , columns=['Family','Age','Fare'])

,Family,Age,Fare
0,1.0,0.75,13.7750
1,1.0,76.00,78.8500
2,0.0,32.00,7.5792
3,1.0,27.00,7.9250
4,1.0,22.00,61.9792
...,...,...,...
259,0.0,20.00,7.2250
260,0.0,27.00,7.8792
261,0.0,43.00,7.8958
262,0.0,16.00,7.6500


1 means travelling with someone
0 means travelling alone

In [23]:
clf = DecisionTreeClassifier()

clf.fit(X_train_trf, y_train)

y_pred2 = clf.predict(X_test_trf)

accuracy_score(y_test, y_pred2)

0.6119402985074627

In [24]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(), X,y,cv=10, scoring='accuracy'))

0.5922459893048129